In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [2]:
!pip install scikit-learn



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## ***Chuẩn bị dữ liệu***


In [4]:
file_path = (r"C:\Users\DUY DUC\OneDrive - National Economics University\Documents\Code workspace\Group6-Project-Python4DS-DSEB-66B\data\Weekly_pivot_with_quarter_season.xlsx")
df = pd.read_excel(file_path)

# Hiển thị preview
print("Dữ liệu gồm:", df.shape[0], "dòng và", df.shape[1], "cột")
print(df.head())


Dữ liệu gồm: 52 dòng và 8 cột
   Week_number  Product_0979  Product_0993  Product_1157  Product_1159  \
0            1          4700             0         10000         50000   
1            2         11800             0        150000         10000   
2            3         16400           100        130000        130000   
3            4          5200             0         40000         30000   
4            5          8000             0         10000         60000   

   Product_1938  Quarter    Season  
0            19        1  mùa Đông  
1            11        1  mùa Đông  
2            10        1  mùa Đông  
3             8        1  mùa Đông  
4            11        1  mùa Đông  


##  ***Tiền xử lí cơ bản***

- Xử lý thiếu (na), kiểu dữ liệu, chuyển Season thành one-hot, để Quarter giữ nguyên (hoặc one-hot).


- Tạo biến thời gian (week index) nếu cần.

In [5]:
try:
    encoder = OneHotEncoder(drop='first', sparse_output=False)
except TypeError:
    encoder = OneHotEncoder(drop='first', sparse=False)

season_enc = encoder.fit_transform(df[['Season']])
season_cols = encoder.get_feature_names_out(['Season'])
season_df = pd.DataFrame(season_enc, columns=season_cols, index=df.index)

# Feature: Quarter + season dummies
X = pd.concat([df[['Quarter']].reset_index(drop=True), season_df.reset_index(drop=True)], axis=1)

# Các cột sản phẩm
product_cols = [c for c in df.columns if c.startswith("Product_")]



## ***Tạo đặc trưng thời gian*** (lag features & rolling)

Thường dữ liệu time-series cần lag để mô hình tuyến tính học được phụ thuộc thời gian.

In [ ]:
products = ["Product_0979","Product_0993","Product_1157","Product_1159","Product_1938"]

# tạo lag 1, lag 2 cho từng product
for p in products:
    data[f"{p}_lag1"] = data[p].shift(1)
    data[f"{p}_lag2"] = data[p].shift(2)
    data[f"{p}_roll4_mean"] = data[p].rolling(window=4, min_periods=1).mean().shift(1)

# drop đầu có NaN do shift
data = data.dropna().reset_index(drop=True)


## ***Chia train/test*** (theo time series hoặc random)

- Nếu dữ liệu tuần thời gian, tốt nhất chia theo thời gian (train cũ → test mới).


- Nếu không có thứ tự mạnh, dùng train_test_split.

In [6]:
n = len(X)
split_idx = int(n * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
print(f"Train: {X_train.shape[0]} hàng | Test: {X_test.shape[0]} hàng")

Train: 41 hàng | Test: 11 hàng


### ***Chuẩn hóa dữ liệu*** ###

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = X_train.select_dtypes(include=['int64','float64']).columns
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])


C:\Users\DUY DUC\AppData\Local\Temp\ipykernel_2668\303616938.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
C:\Users\DUY DUC\AppData\Local\Temp\ipykernel_2668\303616938.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[num_cols] = scaler.transform(X_test[num_cols])


## ***Huấn luyện mô hình Linear Regression đa biến cho từng sản phẩm***

In [8]:
results = []
predictions = {}  # lưu y_test và y_pred để vẽ biểu đồ

for prod in product_cols:
    y = df[prod]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    # Train mô hình Linear Regression
    model = LinearRegression()
    model.fit(X_train, y_train)


## ***Dự đoán & đánh giá***

- Metricts: R², RMSE, MAE cho từng sản phẩm.


- Kiểm tra residuals (độ phân bố phần dư), scatter plot, và histogram.

In [16]:

    # Dự báo
y_pred = model.predict(X_test)

    # Lưu lại để plot
predictions[prod] = pd.DataFrame({
        'Actual': y_test.values,
        'Predicted': y_pred
    }, index=y_test.index)
 # Đánh giá
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5

r2 = r2_score(y_test, y_pred)

    # Lưu hệ số + metric
coeffs = dict(zip(X.columns, model.coef_))
coeffs.update({
        'Intercept': model.intercept_,
        'Product': prod,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    })
results.append(coeffs)
print("✅ Huấn luyện hoàn tất!\n")
results_df = pd.DataFrame(results)
cols_order = ['Product', 'Intercept'] + list(X.columns) + ['MAE', 'RMSE', 'R2']
results_df = results_df[cols_order]

print("📊 Kết quả mô hình Linear Regression:\n")
print(results_df.round(3))

✅ Huấn luyện hoàn tất!

📊 Kết quả mô hình Linear Regression:

        Product  Intercept  Quarter  Season_mùa Thu  Season_mùa Xuân  \
0  Product_1938      8.829   -1.868           0.717           -0.674   
1  Product_1938      8.829   -1.868           0.717           -0.674   
2  Product_1938      8.829   -1.868           0.717           -0.674   

   Season_mùa Đông    MAE    RMSE     R2  
0           -1.314  7.934  10.411 -0.726  
1           -1.314  7.934  10.411 -0.726  
2           -1.314  7.934  10.411 -0.726  
